# prepare data

- train_answer_file: 訓練Annotation File位置
- train_txt_dir: 訓練資料位置
- val_answer_file: 驗證Annotation File位置
- val_txt_dir: 驗證資料位置

In [1]:

train_answer_file = "data/train_answer.txt"
train_txt_dir = "data/train_file"

val_answer_file = "data/val_answer.txt"
val_txt_dir = "data/val_file"


In [2]:
def process_annotation_file(lines):
    entity_dict = {}
    for line in lines:
        items = line.strip('\n').split('\t')
        if len(items) == 5:
            item_list = (int(items[2]),int(items[3]),items[1])
        elif len(items) == 6:
            item_list = (int(items[2]),int(items[3]),items[1])

        file_name = f"{items[0]}.txt"
        if file_name not in entity_dict:
            entity_dict[file_name] = [item_list]
        else:
            entity_dict[file_name].append(item_list)
    return entity_dict

def read_file(path):
    with open(path , 'r' , encoding = 'utf-8-sig') as fr:
        return fr.readlines()

train_entity_dict = process_annotation_file(read_file(train_answer_file))
val_entity_dict = process_annotation_file(read_file(val_answer_file))

In [3]:
import os
def ann_gen(entity_dict, txt_dir):
    ann_list = []
    for k, v in entity_dict.items():
        ann = []
        # print(k)
        file_path = os.path.join(txt_dir,k)
        text_file = open(file_path, "r").read()
        ann.append(text_file)
        ann.append(v)
        ann_list.append(ann)
    return ann_list
train_json = ann_gen(train_entity_dict, train_txt_dir)
val_json = ann_gen(val_entity_dict, val_txt_dir)

輸出成json檔案

In [4]:
import json
json_object = json.dumps(train_json,indent=4)
with open("train.json", "w") as outfile:
    outfile.write(json_object)

json_object = json.dumps(val_json,indent=4)
with open("val.json", "w") as outfile:
    outfile.write(json_object)

# spacy

下載spacy的model

In [12]:
import spacy

# !python -m spacy download en_core_web_trf
# nlp = spacy.load("en_core_web_trf")

!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")



文字轉成token，過濾掉不用的span

In [13]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")


from spacy.util import filter_spans

def create_data(_data, out_name):
    doc_bin = DocBin()
    skip_num = 0
    for training_example in tqdm(_data):
        text = training_example[0]
        labels = training_example[1]
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in labels:
            # span = doc.char_span(start, end, label=label)
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                skip_num+=1
                # print(start, end, label)
            else:
                ents.append(span)
        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents
        doc_bin.add(doc)
    doc_bin.to_disk(f"{out_name}.spacy")
    print(skip_num)



In [14]:
train_data = json.load(open('train.json','r'))

val_data = json.load(open('val.json','r'))

create_data(train_data, "train")
create_data(val_data, "val")

1734
560


100%|██████████| 1734/1734 [00:14<00:00, 120.19it/s]


1391


100%|██████████| 560/560 [00:04<00:00, 117.79it/s]


463


產生config file

In [ ]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

開始training

In [15]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./val.spacy --gpu-id 0

ℹ Saving to output directory: trf
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-12-07 09:17:43,306] [INFO] Set up nlp object from config
[2023-12-07 09:17:43,314] [INFO] Pipeline: ['transformer', 'ner']
[2023-12-07 09:17:43,317] [INFO] Created vocabulary
[2023-12-07 09:17:43,317] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification